In [ ]:
from keras.layers import Input, Dense, Flatten, Reshape, Conv2D, UpSampling2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.callbacks import Callback

import numpy as np
import wandb
from wandb.keras import WandbCallback

run = wandb.init()
config = run.config

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

model = Sequential()
model.add(Reshape((28,28,1), input_shape=(28,28)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
model.add(Reshape((28,28)))

model.compile(optimizer='adam', loss='mse', metrics = ["accuracy"])

model.summary()

# For visualization
class Images(Callback):
      def __init__(self, validation_data):
            self.validation_data = validation_data

      def on_epoch_end(self, epoch, logs):
            indices = np.random.randint(self.validation_data[0].shape[0], size=8)
            test_data = self.validation_data[0][indices]
            pred_data = self.model.predict(test_data)
            wandb.log({
                  "examples": [
                        wandb.Image(np.hstack([data, pred_data[i]]), caption=str(i))
                        for i, data in enumerate(test_data)]},
                  step=epoch)

model.fit(x_train, x_train,
                epochs=5,
                validation_data=(x_test, x_test),
          callbacks=[Images((x_test, x_test)), WandbCallback()])



epoch,1
loss,0.00079
accuracy,0.53328
val_loss,0.00069
val_accuracy,0.5559
_runtime,233
_timestamp,1630697884
_step,1
best_val_loss,0.00069
best_epoch,1


epoch,▁█
loss,█▁
accuracy,▁█
val_loss,█▁
val_accuracy,▁█
_runtime,▁█
_timestamp,▁█
_step,▁█


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 1)         289       
_________________________________________________________________
reshape_3 (Reshape)          (None, 28, 28)           